In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import datetime

## Downloading and preprocessing the data

In [2]:
BUFFER_SIZE = 70_000 # for reshuffling
BATCH_SIZE = 128
NUM_EPOCHS = 20

In [3]:
mnist_dataset, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)

In [4]:
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

In [5]:
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.

    return image, label

In [6]:
train_and_validation_data = mnist_train.map(scale)
test_data = mnist_test.map(scale)

In [7]:
num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

In [8]:
num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

In [9]:
train_and_validation_data = train_and_validation_data.shuffle(BUFFER_SIZE)

In [10]:
train_data = train_and_validation_data.skip(num_validation_samples)
validation_data = train_and_validation_data.take(num_validation_samples)

In [11]:
train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples)

## Creating the model and training it

In [12]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(50, 5, activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Conv2D(50, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)), 
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10)
])

c:\Users\SANDY\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [13]:
model.summary(line_length = 75)

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┓
┃ Layer (type)                   ┃ Output Shape            ┃      Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                │ (None, 24, 24, 50)      │        1,300 │
├────────────────────────────────┼─────────────────────────┼──────────────┤
│ max_pooling2d (MaxPooling2D)   │ (None, 12, 12, 50)      │            0 │
├────────────────────────────────┼─────────────────────────┼──────────────┤
│ conv2d_1 (Conv2D)              │ (None, 10, 10, 50)      │       22,550 │
├────────────────────────────────┼─────────────────────────┼──────────────┤
│ max_pooling2d_1 (MaxPooling2D) │ (None, 5, 5, 50)        │            0 │
├────────────────────────────────┼─────────────────────────┼──────────────┤
│ flatten (Flatten)              │ (None, 1250)            │            0 │
├────────────────────────────────┼─────────────────────────┼──────────────┤
│ dense (Dense)                  │ (None, 10)              │       12,510 │
└────────────────────────────────┴─────────────────────────┴──────────────┘

 Total params: 36,360 (142.03 KB)

 Trainable params: 36,360 (142.03 KB)

 Non-trainable params: 0 (0.00 B)

In [14]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [15]:
model.compile(optimizer='adam', loss=loss_fn, metrics=['accuracy'])

In [16]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor = 'val_loss',
    mode = 'auto',
    min_delta = 0,
    patience = 2,
    verbose = 0, 
    restore_best_weights = True
)

In [17]:
# Logging the training process data to use later in tensorboard
log_dir = "logs\\fit\\" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [18]:
model.fit(
    train_data, 
    epochs = NUM_EPOCHS, 
    callbacks = [tensorboard_callback, early_stopping], 
    validation_data = validation_data,
    verbose = 2
)

Epoch 1/20
422/422 - 10s - 23ms/step - accuracy: 0.9250 - loss: 0.2606 - val_accuracy: 0.9730 - val_loss: 0.0823
Epoch 2/20
422/422 - 8s - 19ms/step - accuracy: 0.9790 - loss: 0.0691 - val_accuracy: 0.9853 - val_loss: 0.0488
Epoch 3/20
422/422 - 7s - 17ms/step - accuracy: 0.9842 - loss: 0.0504 - val_accuracy: 0.9892 - val_loss: 0.0375
Epoch 4/20
422/422 - 7s - 16ms/step - accuracy: 0.9876 - loss: 0.0418 - val_accuracy: 0.9885 - val_loss: 0.0391
Epoch 5/20
422/422 - 7s - 16ms/step - accuracy: 0.9894 - loss: 0.0344 - val_accuracy: 0.9920 - val_loss: 0.0287
Epoch 6/20
422/422 - 7s - 16ms/step - accuracy: 0.9908 - loss: 0.0303 - val_accuracy: 0.9930 - val_loss: 0.0220
Epoch 7/20
422/422 - 7s - 16ms/step - accuracy: 0.9913 - loss: 0.0267 - val_accuracy: 0.9925 - val_loss: 0.0237
Epoch 8/20
422/422 - 7s - 16ms/step - accuracy: 0.9932 - loss: 0.0224 - val_accuracy: 0.9945 - val_loss: 0.0159
Epoch 9/20
422/422 - 7s - 16ms/step - accuracy: 0.9932 - loss: 0.0214 - val_accuracy: 0.9962 - val_loss

## Testing our model

In [19]:
test_loss, test_accuracy = model.evaluate(test_data)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 756ms/step - accuracy: 0.9918 - loss: 0.0242


In [20]:
# Printing the test results
print('Test loss: {0:.4f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

Test loss: 0.0242. Test accuracy: 99.18%


## Visualizing in Tensorboard

In [21]:
%load_ext tensorboard
%tensorboard --logdir "logs/fit"

In [22]:
%reload_ext tensorboard